In [5]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.scheme import IOB1

In [ ]:
import os
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix

import keras
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant
from tensorflow.lookup import KeyValueTensorInitializer, StaticHashTable
from tensorflow.keras.layers import Input, LSTM, Embedding, TimeDistributed, Dropout, Bidirectional, Dense, Layer, InputSpec
from tensorflow_addons.text import crf_log_likelihood, viterbi_decode, crf_decode

import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping


In [3]:
print("Tensorflow Version", tf. __version__)
print("Tensorflow Addons Version", tfa. __version__)
print("Numpy Version", np.__version__)
print("Pandas Version", pd.__version__)
print("Keras Version", keras.__version__)
print("Sklearn Version", sklearn.__version__)

Tensorflow Version 2.10.0
Tensorflow Addons Version 0.21.0
Numpy Version 1.25.2
Pandas Version 1.5.3
Keras Version 2.10.0
Sklearn Version 1.3.0


In [6]:
print("Tensorflow Version", tf. __version__)
print("Tensorflow Addons Version", tfa. __version__)
print("Numpy Version", np.__version__)
print("Pandas Version", pd.__version__)
print("Keras Version", keras.__version__)
print("Sklearn Version", sklearn.__version__)


Tensorflow Version 2.13.1
Tensorflow Addons Version 0.21.0
Numpy Version 1.24.3
Pandas Version 2.0.3
Keras Version 2.13.1
Sklearn Version 1.3.2


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [8]:
data = pd.read_csv('/home/tlf/Documents/mestrado/ner_models/data/df_tokens_labeled_iob.csv', encoding= 'utf-8', index_col=0)
data_train = pd.read_csv('/home/tlf/Documents/mestrado/ner_models/data/df_train_tokens_labeled_iob.csv', encoding= 'utf-8', index_col=0)

# VOCAB

In [9]:
def to_tuples(data):
    iterator = zip(data["token"].values.tolist(),
                  data["iob_label"].values.tolist())
    return [(token, iob_label) for token, iob_label in iterator]

In [10]:
def build_vocab(data):
  all_words = list(set(data["token"].values))
  all_tags = list(set(data["iob_label"].values))

  word2index = {word: idx + 2 for idx, word in enumerate(all_words)}

  word2index["--UNKNOWN_WORD--"] = 0

  word2index["--PADDING--"] = 1

  index2word = {idx: word for word, idx in word2index.items()}

  tag2index = {tag: idx + 1 for idx, tag in enumerate(all_tags)}
  tag2index["--PADDING--"] = 0

  index2tag = {idx: word for word, idx in tag2index.items()}

  return word2index, index2word, tag2index, index2tag

In [18]:
def tokenize(reports, word2index, tag2index):
  contents = []
  labels = []
  for report in reports:
    content = []
    label = []
    for i in range(len(report)):
      token, iob_tag = report[i]
      word_idx = word2index.get(token, 0)
      tag_idx = tag2index.get(iob_tag, 0)
      content.append(word_idx)
      label.append(tag_idx)

    contents.append(content)
    labels.append(label)

  """
  padding the array with max_sentence_size
  pad_sequences(sequences, maxlen=None, dtype="int32", padding="pre", truncating="pre", value=0.0,):
  the maxlen argument if provided, or the length of the longest sequence in the list.
  """

  max_sentence_size = 512
  contents = tf.keras.preprocessing.sequence.pad_sequences(contents, maxlen=max_sentence_size, padding='post', value=1)
  labels = tf.keras.preprocessing.sequence.pad_sequences(labels, maxlen=max_sentence_size, padding='post')


  tag_size = len(tag2index)

  labels_categorical = [tf.keras.utils.to_categorical(i, num_classes=tag_size) for i in labels]
  labels_categorical = np.asarray(labels_categorical)

  return contents, labels, labels_categorical, max_sentence_size

In [13]:
data['iob_label'].isnull().values.sum()

0

In [14]:
word2index, index2word, tag2index, index2tag = build_vocab(data)

In [ ]:
index2tag

In [16]:
reports = data_train.groupby("report").apply(to_tuples).tolist()

In [19]:
text_sequences, tag_sequences, tag_sequences_categorical, max_len = tokenize(reports, word2index, tag2index)

In [20]:
print(len(text_sequences[0]))

512


# Model

In [21]:
def embedding_layer(input_dim, output_dim, input_length, mask_zero):
    return Embedding(input_dim = input_dim, output_dim = output_dim, input_length = input_length, mask_zero = mask_zero)

def bilstm_crf(maxlen, n_tags, lstm_units, embedding_dim, n_words, mask_zero, training = True):
    """
    bilstm_crf - module to build BiLSTM-CRF model
    Inputs:
        - input_shape : tuple
            Tensor shape of inputs, excluding batch size
    Outputs:
        - output : tensorflow.keras.outputs.output
            BiLSTM-CRF output
    """
    input = Input(shape = (maxlen,))
    # Embedding layer
    embeddings = embedding_layer(input_dim = n_words, output_dim = embedding_dim, input_length = maxlen, mask_zero = mask_zero)
    output = embeddings(input)

    # BiLSTM layer
    output = Bidirectional(LSTM(units = lstm_units, return_sequences = True, recurrent_dropout = 0.1))(output)

    # Dense layer
    output = TimeDistributed(Dense(n_tags, activation = 'relu'))(output)

    output = CRF(n_tags, name = 'crf_layer')(output)
    return Model(input, output)

In [22]:
class CRF(Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 transitions=None,
                 **kwargs):
        """
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = transitions

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)


    def get_config(self):
        config = super(CRF, self).get_config()
        config.update({
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'transitions': self.transitions.numpy()  # Convert the transitions to a NumPy array
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        # Since 'transitions' is a NumPy array, we need to convert it back to a tensor
        transitions = tf.convert_to_tensor(config['transitions'])
        # Create a new instance of CRF with the saved configuration
        return cls(output_dim=config['output_dim'], sparse_target=config['sparse_target'], transitions=transitions)

In [23]:
def tag_conf_matrix(cm, tagid):
    tag_name = index2tag[tagid]
    print("Tag name: {}".format(tag_name))
    print(cm[tagid])
    tn, fp, fn, tp = cm[tagid].ravel()
    tag_acc = round( ((tp + tn) / (tn + fp + fn + tp)), 3)


    print("Tag accuracy: {:.3f}\n".format(tag_acc))

In [24]:
def number_to_word_test_sentences_and_tags(index2tag, index2word, X_test, y_test):

    test_sentences= []
    test_tags = []

    for i in range(len(X_test)):
        aux_tag = []

        report = ""
        sentence = X_test[i]
        tags = y_test[i]

        for j in range(len(sentence)):
            word = sentence[j]
            tag = tags[j]
            int_tag = np.where(tag == int(1))
            if str(index2word[word]) != '--PADDING--':
                report = report + " " + str(index2word[word])
                aux_tag.append(index2tag[int(int_tag[0][0])])

        test_sentences.append(report)
        test_tags.append(aux_tag)

    return test_sentences, test_tags

In [24]:
def train_test_df_model(train_sentences, train_tags):

    train_df = pd.DataFrame(columns = ['report', 'word', 'tag'])

    for i in range (len(train_sentences)):
        sentence = train_sentences[i]
        tags = train_tags[i]
        
        sentence = sentence.split()

        if i < 10:
            retval = ""
            for w, t in zip(sentence, tags):
                retval = retval + "{:25}: {:10}".format(w, t) + "\n"
                aux_dict = {'report': ('report_0' + str(i)), 'word': w, 'tag' : t}
                df_new_row = pd.DataFrame([aux_dict])
                train_df = pd.concat([train_df, df_new_row])

        else:
            retval = ""
            for w, t in zip(sentence, tags):
                retval = retval + "{:25}: {:10}".format(w, t) + "\n"
                aux_dict = {'report': ('report_' + str(i)), 'word': w, 'tag' : t}
                df_new_row = pd.DataFrame([aux_dict])
                train_df = pd.concat([train_df, df_new_row])

    return train_df

In [25]:
def result_df_model_previous(test_sentences, test_tags, model, word2index, index2tag, MAX_SENTENCE):

    test_df = pd.DataFrame(columns = ['report', 'word', 'tag', 'tag_pred'])

    for i in range (len(test_sentences)):

        tags = test_tags[i]
        
        sentence = sentence.split()
        padded_sentence = sentence + [word2index["--PADDING--"]] * (MAX_SENTENCE - len(sentence))
        padded_sentence = [word2index.get(w, 0) for w in padded_sentence]

        pred = model.predict(np.array([padded_sentence]))
        pred = np.argmax(pred, axis=-1)

        if i < 10:
            retval = ""
            for w, t, p in zip(sentence, tags, pred[0]):
                retval = retval + "{:25}: {:10} {:5}".format(w, t, index2tag[p]) + "\n"
                aux_dict = {'report': ('report_0' + str(i)), 'word': w, 'tag' : t, 'tag_pred' : index2tag[p]}
                df_new_row = pd.DataFrame([aux_dict])
                test_df = pd.concat([test_df, df_new_row])

        else:
            retval = ""
            for w, t, p in zip(sentence, tags, pred[0]):
                retval = retval + "{:25}: {:10} {:5}".format(w, t, index2tag[p]) + "\n"
                aux_dict = {'report': ('report_' + str(i)), 'word': w, 'tag' : t, 'tag_pred' : index2tag[p]}
                df_new_row = pd.DataFrame([aux_dict])
                test_df = pd.concat([test_df, df_new_row])

    return test_df

In [26]:
X_train = text_sequences
y_train = tag_sequences_categorical
train_sentences, train_tags = number_to_word_test_sentences_and_tags(index2tag, index2word, text_sequences, tag_sequences_categorical)

In [ ]:
train_sentences[0]

In [ ]:
text_sequences[0]

In [ ]:
train_tags[0]

In [ ]:
tag_sequences_categorical[0]

# CREATE MODELS

In [30]:
LSTM_DROPOUT = 0.1
MAX_EPOCHS = 10
lr = 0.01

In [31]:
DENSE_EMBEDDING_LIST = [25, 100, 300]
LSTM_UNITS_LIST = [25, 50, 100]
BATCH_SIZE_LIST = [4, 8, 16]

In [32]:
n_words = len(word2index)
n_tags = len(tag2index)
lstm_dropout = LSTM_DROPOUT
epochs = MAX_EPOCHS

## MODEL 00

In [33]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_00 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_00.summary()

# compile model
model_00.compile(optimizer = Adam(learning_rate = lr), loss = model_00.layers[-1].loss, metrics = model_00.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)

"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_00.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 1

In [35]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_01 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_01.summary()

# compile model
model_01.compile(optimizer = Adam(learning_rate = lr), loss = model_01.layers[-1].loss, metrics = model_01.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)

"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_01.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 02

In [37]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_02 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_02.summary()

# compile model
model_02.compile(optimizer = Adam(learning_rate = lr), loss = model_02.layers[-1].loss, metrics = model_02.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)

"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_02.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 03

In [39]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_03 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_03.summary()

# compile model
model_03.compile(optimizer = Adam(learning_rate = lr), loss = model_03.layers[-1].loss, metrics = model_03.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_03.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 04

In [41]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_04 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_04.summary()

# compile model
model_04.compile(optimizer = Adam(learning_rate = lr), loss = model_04.layers[-1].loss, metrics = model_04.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_04.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 05

In [43]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_05 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_05.summary()

# compile model
model_05.compile(optimizer = Adam(learning_rate = lr), loss = model_05.layers[-1].loss, metrics = model_05.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_05.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 06

In [45]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_06 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_06.summary()

# compile model
model_06.compile(optimizer = Adam(learning_rate = lr), loss = model_06.layers[-1].loss, metrics = model_06.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_06.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 07

In [47]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_07 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_07.summary()

# compile model
model_07.compile(optimizer = Adam(learning_rate = lr), loss = model_07.layers[-1].loss, metrics = model_07.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_07.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 08

In [49]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_08 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_08.summary()

# compile model
model_08.compile(optimizer = Adam(learning_rate = lr), loss = model_08.layers[-1].loss, metrics = model_08.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_08.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 09

In [51]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_09 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_09.summary()

# compile model
model_09.compile(optimizer = Adam(learning_rate = lr), loss = model_09.layers[-1].loss, metrics = model_09.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_09.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 10


In [53]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_10 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_10.summary()

# compile model
model_10.compile(optimizer = Adam(learning_rate = lr), loss = model_10.layers[-1].loss, metrics = model_10.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_10.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 11

In [55]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_11 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_11.summary()

# compile model
model_11.compile(optimizer = Adam(learning_rate = lr), loss = model_11.layers[-1].loss, metrics = model_11.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_11.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 12

In [57]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_12 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_12.summary()

# compile model
model_12.compile(optimizer = Adam(learning_rate = lr), loss = model_12.layers[-1].loss, metrics = model_12.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_12.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 13

In [59]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_13 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_13.summary()

# compile model
model_13.compile(optimizer = Adam(learning_rate = lr), loss = model_13.layers[-1].loss, metrics = model_13.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_13.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 14

In [61]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_14 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_14.summary()

# compile model
model_14.compile(optimizer = Adam(learning_rate = lr), loss = model_14.layers[-1].loss, metrics = model_14.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_14.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 15

In [63]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_15 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_15.summary()

# compile model
model_15.compile(optimizer = Adam(learning_rate = lr), loss = model_15.layers[-1].loss, metrics = model_15.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_15.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 16

In [65]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_16 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_16.summary()

# compile model
model_16.compile(optimizer = Adam(learning_rate = lr), loss = model_16.layers[-1].loss, metrics = model_16.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_16.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 17

In [67]:
embed_dim = DENSE_EMBEDDING_LIST[1]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_17 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_17.summary()

# compile model
model_17.compile(optimizer = Adam(learning_rate = lr), loss = model_17.layers[-1].loss, metrics = model_17.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_17.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 18

In [69]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_18 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_18.summary()

# compile model
model_18.compile(optimizer = Adam(learning_rate = lr), loss = model_18.layers[-1].loss, metrics = model_18.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_18.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 19

In [71]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_19 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_19.summary()

# compile model
model_19.compile(optimizer = Adam(learning_rate = lr), loss = model_19.layers[-1].loss, metrics = model_19.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_19.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 20

In [73]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_20 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_20.summary()

# compile model
model_20.compile(optimizer = Adam(learning_rate = lr), loss = model_20.layers[-1].loss, metrics = model_20.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_20.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 21

In [75]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_21 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_21.summary()

# compile model
model_21.compile(optimizer = Adam(learning_rate = lr), loss = model_21.layers[-1].loss, metrics = model_21.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_21.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 22

In [77]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_22 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_22.summary()

# compile model
model_22.compile(optimizer = Adam(learning_rate = lr), loss = model_22.layers[-1].loss, metrics = model_22.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_22.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 23

In [79]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[1]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_23 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_23.summary()

# compile model
model_23.compile(optimizer = Adam(learning_rate = lr), loss = model_23.layers[-1].loss, metrics = model_23.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_23.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 24

In [81]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[0]

In [ ]:
# create model
model_24 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_24.summary()

# compile model
model_24.compile(optimizer = Adam(learning_rate = lr), loss = model_24.layers[-1].loss, metrics = model_24.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_24.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 25

In [83]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[1]

In [ ]:
# create model
model_25 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_25.summary()

# compile model
model_25.compile(optimizer = Adam(learning_rate = lr), loss = model_25.layers[-1].loss, metrics = model_25.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_25.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Model 26

In [85]:
embed_dim = DENSE_EMBEDDING_LIST[2]
lstm_unit = LSTM_UNITS_LIST[2]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
# create model
model_26 = bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)
model_26.summary()

# compile model
model_26.compile(optimizer = Adam(learning_rate = lr), loss = model_26.layers[-1].loss, metrics = model_26.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)
"""
steps_per_epoch: Integer or None. Total number of steps (batches of samples) before declaring
one epoch finished and starting the next epoch. When training with input tensors such as
TensorFlow data tensors, the default None is equal to the number of samples in your dataset
divided by the batch size
"""
#steps_per_epoch = x_train_len / 4

model_26.fit(X_train, y_train, epochs = epochs, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

## Resultados

In [87]:
data_test = pd.read_csv('/home/tlf/Documents/mestrado/ner_models/data/df_test_tokens_labeled_iob.csv', encoding= 'utf-8', index_col=0)
reports = data_test.groupby("report").apply(to_tuples).tolist()
test_text_sequences, test_tag_sequences, test_tag_sequences_categorical, max_len = tokenize(reports, word2index, tag2index)

X_test = test_text_sequences
y_test = test_tag_sequences_categorical

test_sentences, test_tags = number_to_word_test_sentences_and_tags(index2tag, index2word, test_text_sequences, test_tag_sequences_categorical)

In [ ]:
result_df_model_00 = result_df_model_previous(test_sentences, test_tags, model_00, word2index, index2tag, max_len)
result_df_model_00.to_csv("result_df_model_00.csv", encoding='utf-8')

result_df_model_01 = result_df_model_previous(test_sentences, test_tags, model_01, word2index, index2tag, max_len)
result_df_model_01.to_csv("result_df_model_01.csv", encoding='utf-8')

result_df_model_02 = result_df_model_previous(test_sentences, test_tags, model_02, word2index, index2tag, max_len)
result_df_model_02.to_csv("result_df_model_02.csv", encoding='utf-8')

result_df_model_03 = result_df_model_previous(test_sentences, test_tags, model_03, word2index, index2tag, max_len)
result_df_model_03.to_csv("result_df_model_03.csv", encoding='utf-8')

result_df_model_04 = result_df_model_previous(test_sentences, test_tags, model_04, word2index, index2tag, max_len)
result_df_model_04.to_csv("result_df_model_04.csv", encoding='utf-8')

result_df_model_05 = result_df_model_previous(test_sentences, test_tags, model_05, word2index, index2tag, max_len)
result_df_model_05.to_csv("result_df_model_05.csv", encoding='utf-8')

result_df_model_06 = result_df_model_previous(test_sentences, test_tags, model_06, word2index, index2tag, max_len)
result_df_model_06.to_csv("result_df_model_06.csv", encoding='utf-8')

result_df_model_07 = result_df_model_previous(test_sentences, test_tags, model_07, word2index, index2tag, max_len)
result_df_model_07.to_csv("result_df_model_07.csv", encoding='utf-8')

result_df_model_08 = result_df_model_previous(test_sentences, test_tags, model_08, word2index, index2tag, max_len)
result_df_model_08.to_csv("result_df_model_08.csv", encoding='utf-8')

result_df_model_09 = result_df_model_previous(test_sentences, test_tags, model_09, word2index, index2tag, max_len)
result_df_model_09.to_csv("result_df_model_09.csv", encoding='utf-8')

result_df_model_10 = result_df_model_previous(test_sentences, test_tags, model_10, word2index, index2tag, max_len)
result_df_model_10.to_csv("result_df_model_10.csv", encoding='utf-8')

result_df_model_11 = result_df_model_previous(test_sentences, test_tags, model_11, word2index, index2tag, max_len)
result_df_model_11.to_csv("result_df_model_11.csv", encoding='utf-8')

result_df_model_12 = result_df_model_previous(test_sentences, test_tags, model_12, word2index, index2tag, max_len)
result_df_model_12.to_csv("result_df_model_12.csv", encoding='utf-8')

result_df_model_13 = result_df_model_previous(test_sentences, test_tags, model_13, word2index, index2tag, max_len)
result_df_model_13.to_csv("result_df_model_13.csv", encoding='utf-8')

result_df_model_14 = result_df_model_previous(test_sentences, test_tags, model_14, word2index, index2tag, max_len)
result_df_model_14.to_csv("result_df_model_14.csv", encoding='utf-8')

result_df_model_15 = result_df_model_previous(test_sentences, test_tags, model_15, word2index, index2tag, max_len)
result_df_model_15.to_csv("result_df_model_15.csv", encoding='utf-8')

result_df_model_16 = result_df_model_previous(test_sentences, test_tags, model_16, word2index, index2tag, max_len)
result_df_model_16.to_csv("result_df_model_16.csv", encoding='utf-8')

result_df_model_17 = result_df_model_previous(test_sentences, test_tags, model_17, word2index, index2tag, max_len)
result_df_model_17.to_csv("result_df_model_17.csv", encoding='utf-8')

result_df_model_18 = result_df_model_previous(test_sentences, test_tags, model_18, word2index, index2tag, max_len)
result_df_model_18.to_csv("result_df_model_18.csv", encoding='utf-8')

result_df_model_19 = result_df_model_previous(test_sentences, test_tags, model_19, word2index, index2tag, max_len)
result_df_model_19.to_csv("result_df_model_19.csv", encoding='utf-8')

result_df_model_20 = result_df_model_previous(test_sentences, test_tags, model_20, word2index, index2tag, max_len)
result_df_model_20.to_csv("result_df_model_20.csv", encoding='utf-8')

result_df_model_21 = result_df_model_previous(test_sentences, test_tags, model_21, word2index, index2tag, max_len)
result_df_model_21.to_csv("result_df_model_21.csv", encoding='utf-8')

result_df_model_22 = result_df_model_previous(test_sentences, test_tags, model_22, word2index, index2tag, max_len)
result_df_model_22.to_csv("result_df_model_22.csv", encoding='utf-8')

result_df_model_23 = result_df_model_previous(test_sentences, test_tags, model_23, word2index, index2tag, max_len)
result_df_model_23.to_csv("result_df_model_23.csv", encoding='utf-8')

result_df_model_24 = result_df_model_previous(test_sentences, test_tags, model_24, word2index, index2tag, max_len)
result_df_model_24.to_csv("result_df_model_24.csv", encoding='utf-8')

result_df_model_25 = result_df_model_previous(test_sentences, test_tags, model_25, word2index, index2tag, max_len)
result_df_model_25.to_csv("result_df_model_25.csv", encoding='utf-8')

result_df_model_26 = result_df_model_previous(test_sentences, test_tags, model_26, word2index, index2tag, max_len)
result_df_model_26.to_csv("result_df_model_26.csv", encoding='utf-8')

## Save Models

In [ ]:
model_00.save("model_00")
model_01.save("model_01")
model_02.save("model_02")
model_03.save("model_03")
model_04.save("model_04")
model_05.save("model_05")
model_06.save("model_06")
model_07.save("model_07")
model_08.save("model_08")
model_09.save("model_09")
model_10.save("model_10")
model_11.save("model_11")
model_12.save("model_12")
model_13.save("model_13")
model_14.save("model_14")
model_15.save("model_15")
model_16.save("model_16")
model_17.save("model_17")
model_18.save("model_18")
model_19.save("model_19")
model_20.save("model_20")
model_21.save("model_21")
model_22.save("model_22")
model_23.save("model_23")
model_24.save("model_24")
model_25.save("model_25")
model_26.save("model_26")


## Save Models - TF

In [ ]:
"""model_00.save("model_00.tf")
model_01.save("model_01.tf")
model_02.save("model_02.tf")
model_03.save("model_03.tf")
model_04.save("model_04.tf")
model_05.save("model_05.tf")
model_06.save("model_06.tf")
model_07.save("model_07.tf")
model_08.save("model_08.tf")
model_09.save("model_09.tf")
model_10.save("model_10.tf")
model_11.save("model_11.tf")
model_12.save("model_12.tf")
model_13.save("model_13.tf")
model_14.save("model_14.tf")
model_15.save("model_15.tf")
model_16.save("model_16.tf")
model_17.save("model_17.tf")
model_18.save("model_18.tf")
model_19.save("model_19.tf")
model_20.save("model_20.tf")
model_21.save("model_21.tf")
model_22.save("model_22.tf")
model_23.save("model_23.tf")
model_24.save("model_24.tf")
model_25.save("model_25.tf")
model_26.save("model_26.tf")"""

## Save Models - H5

In [87]:
"""model_00.save("model_00.h5")
model_01.save("model_01.h5")
model_02.save("model_02.h5")
model_03.save("model_03.h5")
model_04.save("model_04.h5")
model_05.save("model_05.h5")
model_06.save("model_06.h5")
model_07.save("model_07.h5")
model_08.save("model_08.h5")
model_09.save("model_09.h5")
model_10.save("model_10.h5")
model_11.save("model_11.h5")
model_12.save("model_12.h5")
model_13.save("model_13.h5")
model_14.save("model_14.h5")
model_15.save("model_15.h5")
model_16.save("model_16.h5")
model_17.save("model_17.h5")
model_18.save("model_18.h5")
model_19.save("model_19.h5")
model_20.save("model_20.h5")
model_21.save("model_21.h5")
model_22.save("model_22.h5")
model_23.save("model_23.h5")
model_24.save("model_24.h5")
model_25.save("model_25.h5")
model_26.save("model_26.h5")"""

## Avaliação


In [90]:
def to_tuples(data):
    iterator = zip(data["word"].values.tolist(),
                  data["tag"].values.tolist(),
                  data["tag_pred"].values.tolist())
    return [(word, tag, tag_pred) for word, tag, tag_pred in iterator]

def tuple_2_list(sentences):
    texts = [[word[0] for word in sentence] for sentence in sentences]
    tags = [[word[1] for word in sentence] for sentence in sentences]
    tags_pred = [[word[2] for word in sentence] for sentence in sentences]

    return texts, tags, tags_pred

In [91]:
all_tags = ['O',
'ACH',
'ATE',
'BORDAS',
'CAL',
'LOC',
'TAM',
]

In [96]:
current_directory = os.getcwd()+"/results"

In [97]:
filenames = os.listdir(current_directory)
filenames.sort()

In [ ]:
print(filenames)

#### Previous Eval

In [ ]:
model_name = []
results_by_model = []
results_by_model_by_tag = []

for filename in filenames:
    f = os.path.join(current_directory, filename)
    if os.path.isfile(f):
        print(f)

    data = pd.read_csv(f, index_col=0)

    sentences = data.groupby("report").apply(to_tuples).tolist()
    texts, tags, tags_pred = tuple_2_list(sentences)
    
    # Avaliação
    evaluator = Evaluator(tags, tags_pred, tags=all_tags, loader="list")

    # Returns overall metrics and metrics for each tag
    results, results_by_tag = evaluator.evaluate()
 
    model_name.append(filename)
    results_by_model.append(results)
    results_by_model_by_tag.append(results_by_tag)


## Metrics Best Model

In [97]:
metrics_model_20_df = pd.DataFrame(results_by_model[20])

metrics_localizacao_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Localização"])

metrics_calcificacao_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Calcificação"])

metrics_achado_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Achado"])

metrics_bordas_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Bordas"])

metrics_atenuacao_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Atenuação"])

metrics_tamanho_model_20_df = pd.DataFrame(results_by_model_by_tag[20]["Tamanho"])

In [98]:
metrics_model_20_df.to_csv("metrics_model_20.csv")
metrics_localizacao_model_20_df.to_csv("metrics_localizacao_model_20.csv")
metrics_calcificacao_model_20_df.to_csv("metrics_calcificacao_model_20.csv")
metrics_achado_model_20_df.to_csv("metrics_achado_model_20.csv")
metrics_bordas_model_20_df.to_csv("metrics_bordas_model_20.csv")
metrics_atenuacao_model_20_df.to_csv("metrics_atenuacao_model_20.csv")
metrics_tamanho_model_20_df.to_csv("metrics_tamanho_model_20.csv")

In [99]:
best_model_tags_ent_type = []
best_model_tags_partial = []
best_model_tags_strict = []
best_model_tags_exact = []

best_model = results_by_model_by_tag[20]

print(best_model)

for entity in all_tags:
    tag_ent_type = []
    tag_partial = []
    tag_strict = []
    tag_exact = []

    entity = str(entity)
    # entity_type
    ent_precision = round(best_model[entity]["ent_type"]["precision"], 2)
    ent_recall = round(best_model[entity]["ent_type"]["recall"], 2)
    ent_f1 = round(best_model[entity]["ent_type"]["f1"], 2)
    tag_ent_type.append(ent_precision)
    tag_ent_type.append(ent_recall)
    tag_ent_type.append(ent_f1)

    # partial
    partial_precision = round(best_model[entity]["partial"]["precision"], 2)
    partial_recall = round(best_model[entity]["partial"]["recall"], 2)
    partial_f1 = round(best_model[entity]["partial"]["f1"], 2)
    tag_partial.append(partial_precision)
    tag_partial.append(partial_recall)
    tag_partial.append(partial_f1)

    # strict
    strict_precision = round(best_model[entity]["strict"]["precision"], 2)
    strict_recall = round(best_model[entity]["strict"]["recall"], 2)
    strict_f1 = round(best_model[entity]["strict"]["f1"], 2)
    tag_strict.append(strict_precision)
    tag_strict.append(strict_recall)
    tag_strict.append(strict_f1)

    # exact
    exact_precision = round(best_model[entity]["exact"]["precision"], 2)
    exact_recall = round(best_model[entity]["exact"]["recall"], 2)
    exact_f1 = round(best_model[entity]["exact"]["f1"], 2)
    tag_exact.append(exact_precision)
    tag_exact.append(exact_recall)
    tag_exact.append(exact_f1)

    best_model_tags_ent_type.append(tag_ent_type)
    best_model_tags_partial.append(tag_partial)
    best_model_tags_strict.append(tag_strict)
    best_model_tags_exact.append(tag_exact)

{'O': {'ent_type': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'partial': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'strict': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}, 'exact': {'correct': 0, 'incorrect': 0, 'partial': 0, 'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0, 'f1': 0}}, 'Achado': {'ent_type': {'correct': 120, 'incorrect': 0, 'partial': 0, 'missed': 15, 'spurious': 6, 'possible': 135, 'actual': 126, 'precision': 0.9523809523809523, 'recall': 0.8888888888888888, 'f1': 0.9195402298850575}, 'partial': {'correct': 116, 'incorrect': 0, 'partial': 4, 'missed': 15, 'spurious': 6, 'possible': 135, 'actual': 126, 'precision': 0.9365079365079365, 'recall': 0.87407

In [100]:
best_model_tags_ent_type_np = np.array(best_model_tags_ent_type)
best_model_tags_partial_np = np.array(best_model_tags_partial)
best_model_tags_strict_np = np.array(best_model_tags_strict)
best_model_tags_exact_np = np.array(best_model_tags_exact)

In [101]:
ent_type_results_best_model_df = pd.DataFrame(best_model_tags_ent_type_np, columns = ["Precision", "Recall", "F1"])
partial_results_best_model_df = pd.DataFrame(best_model_tags_partial_np, columns = ["Precision", "Recall", "F1"])
strict_results_best_model_df = pd.DataFrame(best_model_tags_strict_np, columns = ["Precision", "Recall", "F1"])
exact_results_best_model_df = pd.DataFrame(best_model_tags_exact_np, columns = ["Precision", "Recall", "F1"])

In [102]:
ent_type_results_best_model_df.to_csv("ent_type_results_best_model_df.csv", encoding="utf-8")
partial_results_best_model_df.to_csv("partial_results_best_model_df.csv", encoding="utf-8")
strict_results_best_model_df.to_csv("strict_results_best_model_df.csv", encoding="utf-8")
exact_results_best_model_df.to_csv("exact_results_best_model_df.csv", encoding="utf-8")

## Avaliar os Modelos

In [93]:
models_ent_type = []
models_partial = []
models_strict = []
models_exact = []

for result_model in results_by_model:
    ent_type = []
    partial = []
    strict = []
    exact = []

    # entity_type
    ent_precision = round(result_model["ent_type"]["precision"], 2)
    ent_recall = round(result_model["ent_type"]["recall"], 2)
    ent_f1 = round(result_model["ent_type"]["f1"], 2)
    ent_type.append(ent_precision)
    ent_type.append(ent_recall)
    ent_type.append(ent_f1)

    # partial
    partial_precision = round(result_model["partial"]["precision"], 2)
    partial_recall = round(result_model["partial"]["recall"], 2)
    partial_f1 = round(result_model["partial"]["f1"], 2)
    partial.append(partial_precision)
    partial.append(partial_recall)
    partial.append(partial_f1)

    # strict
    strict_precision = round(result_model["strict"]["precision"], 2)
    strict_recall = round(result_model["strict"]["recall"], 2)
    strict_f1 = round(result_model["strict"]["f1"], 2)
    strict.append(strict_precision)
    strict.append(strict_recall)
    strict.append(strict_f1)

    # exact
    exact_precision = round(result_model["exact"]["precision"], 2)
    exact_recall = round(result_model["exact"]["recall"], 2)
    exact_f1 = round(result_model["exact"]["f1"], 2)
    exact.append(exact_precision)
    exact.append(exact_recall)
    exact.append(exact_f1)


    models_ent_type.append(ent_type)
    models_partial.append(partial)
    models_strict.append(strict)
    models_exact.append(exact)

In [94]:
models_ent_type_np = np.array(models_ent_type)
models_partial_np = np.array(models_partial)
models_strict_np = np.array(models_strict)
models_exact_np = np.array(models_exact)

In [95]:
ent_type_results_by_model_df = pd.DataFrame(models_ent_type_np, columns = ["Precision", "Recall", "F1"])
partial_results_by_model_df = pd.DataFrame(models_partial_np, columns = ["Precision", "Recall", "F1"])
strict_results_by_model_df = pd.DataFrame(models_strict_np, columns = ["Precision", "Recall", "F1"])
exact_results_by_model_df = pd.DataFrame(models_exact_np, columns = ["Precision", "Recall", "F1"])

In [96]:
ent_type_results_by_model_df.to_csv("ent_type_results_by_model_df.csv", encoding="utf-8")
partial_results_by_model_df.to_csv("partial_results_by_model_df.csv", encoding="utf-8")
strict_results_by_model_df.to_csv("strict_results_by_model_df.csv", encoding="utf-8")
exact_results_by_model_df.to_csv("exact_results_by_model_df.csv", encoding="utf-8")

# Carregar o modelo

In [112]:
cwd = 'c:\\Users\\tarci\\Desktop\\modelos_mestrado\\bilstm_antigo\\models'

In [24]:
embed_dim = DENSE_EMBEDDING_LIST[0]
lstm_unit = LSTM_UNITS_LIST[0]
batch_size = BATCH_SIZE_LIST[2]

In [ ]:
model_test =  bilstm_crf(maxlen = max_len, n_tags = n_tags, lstm_units = lstm_unit, embedding_dim = embed_dim, n_words = n_words, mask_zero = True)

# compile model
model_test.compile(optimizer = Adam(learning_rate = lr), loss = model_test.layers[-1].loss, metrics = model_test.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)

model_test.fit(X_train, y_train, epochs = 1, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

In [ ]:
model_test.save("model_teste_load")

Necessário passar essas funções 

In [57]:
def viterbi_accuracy(y_true, y_pred):
    mask = K.cast(K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
    shape = tf.shape(y_pred)
    sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
    y_pred, _ = crf_decode(y_pred, K.zeros_like(y_pred), sequence_lengths)
    if K.ndim(y_true) == K.ndim(y_pred) - 1:
        y_true = K.expand_dims(y_true, K.ndim(y_pred) - 1)
    y_pred = K.cast(y_pred, 'int32')
    y_true = K.cast(y_true, 'int32')
    corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
    return K.sum(corrects * mask) / K.sum(mask)

In [58]:
def crf_loss(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred, dtype=CRF(dtype='float32').dtype)
    log_likelihood, _ = tf.keras.layers.CRF(dtype='float32')(y_pred, y_true)
    return tf.reduce_mean(-log_likelihood)

In [59]:
loaded_model = tf.keras.models.load_model('model_teste_load', custom_objects={'CRF': CRF, 'viterbi_accuracy': viterbi_accuracy, 'crf_loss': crf_loss})

In [ ]:
# compile model
loaded_model.compile(optimizer = Adam(learning_rate = lr), loss = loaded_model.layers[-1].loss, metrics = loaded_model.layers[-1].accuracy)

# train model
reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 5, verbose = 1)
early_stopping = EarlyStopping(monitor = 'loss', min_delta = 0, patience = 5, verbose = 1)

loaded_model.fit(X_train, y_train, epochs = 1, initial_epoch = 0, callbacks = [early_stopping, reduce_lr], verbose = 1, shuffle = True)

#### Seq Eval

In [ ]:
model_name = []
results_by_model = []
results_by_model_by_tag = []

for filename in filenames:
    f = os.path.join(current_directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

    data = pd.read_csv(f, index_col=0)
    sentences = data.groupby("report").apply(to_tuples).tolist()
    texts, tags, tags_pred = tuple_2_list(sentences)
    
    print()
    print()
    print(filename)
    result_dict = classification_report(tags, tags_pred, mode="strict", scheme=IOB2, zero_division=False)
    print(result_dict)
    print()
    print()

    if filename == "result_df_model_13.csv":
        best_result = result_dict



In [117]:
import json

In [ ]:
res = json.loads(best_result)